## Initial data
Initial constants and derived data. Assumes ender pearl is in 1st quadrant

In [ ]:
# Allowed coordinate of wind charge (1-axis, including mirrored copies)
windChargeCoord = [
0.03125,
0.08375,
0.09375,
0.15625,
0.16625,
0.20875,
0.27125,
0.72875,
0.79125,
0.83375,
0.84375,
0.90625,
0.91625,
0.96875,
]

# Allowed coordinate of pearl (1-axis)
enderPearlCoord = [
    0.0625,
    # 0.24,
    # 0.3025,
]

# Max distance from pearl to axis to count them as collinear (units - blocks)
collinearDistThreshold = 1E-6
# Max error in distance ratio to count pearl as being in center of axis (units - fraction)
axisCenterRatioError = 0.01
# Max angle error between two axis from 90 deg (units - deg)
axisOrthogErrorDeg = 0.1
# Max difference between two axis lengths (absolute)
axisLengthError = 0.001
# Minimum safe distance to avoid pearl colliding with charge (along one axis)
axisMinDistThreshold = (0.3125 + 0.25) / 2

from itertools import product

# List of all possible wind charge coords, for all 4 quadrants in order (1st..4th -> 0..3)
windChargePosQuads = []
windChargePosQuads.append(list(product(windChargeCoord, repeat=2)))
windChargePosQuads.append([(-x,  y) for (x, y) in windChargePosQuads[0]])
windChargePosQuads.append([(-x, -y) for (x, y) in windChargePosQuads[0]])
windChargePosQuads.append([( x, -y) for (x, y) in windChargePosQuads[0]])

# List of all possible ender pearl positions
enderPearlPosArr = list(product(windChargeCoord, repeat=2))

## Shared functions

In [2]:
import numpy as np

from itertools import product
from math import hypot

# (caution, made by ai)
def pointToLineDistance(P, A, B):
    x0, y0 = P
    x1, y1 = A
    x2, y2 = B
    numerator = abs((x2 - x1)*(y1 - y0) - (x1 - x0)*(y2 - y1))
    denominator = hypot(x2 - x1, y2 - y1)
    return numerator / denominator

def pointToPointDist(A, B):
    x1, y1 = A
    x2, y2 = B
    return hypot(x2 - x1, y2 - y1)

# Try all combinations of wind charge and ender pearl positions
def getValidAxisList(posAList, posBlist, targetList):
    
    validCombos = []
    for pointCombo in product(posAList, posBlist, targetList):
        (chargeAPos, chargeBPos, pearlPos) = pointCombo
        colinError = pointToLineDistance(pearlPos, chargeAPos, chargeBPos)
        if abs(colinError) < abs(collinearDistThreshold):
            distAP = pointToPointDist(chargeAPos, pearlPos)
            distBP = pointToPointDist(chargeBPos, pearlPos)
            distRatio = distAP / distBP
            if (distRatio <= (1.0 + axisCenterRatioError)) and (distRatio >= (1.0 - axisCenterRatioError)):
                # Found pair of values with pearl in the center between them
                validCombos.append(pointCombo)
    
    return validCombos

# (caution - made by ai)
def angleBetweenLinesDeg(A, B, C, D):
    # Convert points to vectors
    u = np.array(B) - np.array(A)
    v = np.array(D) - np.array(C)

    # Compute dot product and magnitudes
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)

    # Compute angle in radians and convert to degrees
    cos_theta = dot_product / (norm_u * norm_v)
    angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip for numerical stability
    angle_deg = np.degrees(angle_rad)

    return angle_deg

## Axis search
### First axis ("X")
Find all point pairs that cross ender pearl position. Points are in 1st and 3rd quadrant

In [3]:
validAxis1 = getValidAxisList(windChargePosQuads[0], windChargePosQuads[2], enderPearlPosArr)

### Second axis ("Y")
Same for 2nd and 4th quadrants

In [4]:
validAxis2 = getValidAxisList(windChargePosQuads[1], windChargePosQuads[3], enderPearlPosArr)

## Axis matching
Need to find matching pairs of axis

In [5]:
from itertools import product
from math import atan2, degrees

# Returns true if pearl and charge will collide
def testHitboxCollision(A, B):
    dx = abs(A[0] - B[0])
    dy = abs(A[1] - B[1])
    return (dx <= axisMinDistThreshold) or (dy <= axisMinDistThreshold)

axisCandidates = []
for axisPair in product(validAxis1, validAxis2):
    (axis1, axis2) = axisPair
    pearl1 = axis1[2]
    pearl2 = axis2[2]
    if pearl1 == pearl2:
        # See if axis are 90 deg apart
        axisAngleDeg = angleBetweenLinesDeg(axis1[0], axis1[1], axis2[0], axis2[1])
        if abs((abs(axisAngleDeg) - 90)) < axisOrthogErrorDeg:
            # Axis are pretty orthogonal. Compare lengths
            axisALen = pointToPointDist(axis1[0], axis1[1])
            axisBLen = pointToPointDist(axis2[0], axis2[1])
            if abs(axisALen - axisBLen) < axisLengthError:
                # Axis are close in length. Check for hitbox collision
                haveCollision = \
                    testHitboxCollision(pearl1, axis1[0]) or \
                    testHitboxCollision(pearl1, axis1[1]) or \
                    testHitboxCollision(pearl1, axis2[0]) or \
                    testHitboxCollision(pearl1, axis1[1])
                if not haveCollision:
                    axisCandidates.append(axisPair)


## Graphical wrapper


In [6]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
from tabulate import tabulate

def plotResult(index):
    fig = plt.figure(figsize=(6, 6))
    
    # First plot graphical data
    plotSFig = fig.add_subplot()
    axisPair = axisCandidates[index]
    for (windA, windB, pearl) in axisPair:
        xVals = (windA[0], windB[0])
        yVals = (windA[1], windB[1])
        # Plot solid black line, point marker
        plotSFig.plot(xVals, yVals, ".-k", linewidth=1)
        # Plot ender pearl (red plus), every time just in case
        plotSFig.plot(pearl[0], pearl[1], "+r")
    plotSFig.axhline(0, color="lightgray")
    plotSFig.axvline(0, color="lightgray")
    plotSFig.set_title(f"Total matches: {len(axisCandidates)}")
    plotSFig.set_xlabel("X")
    plotSFig.set_ylabel("Z", rotation=0)
    # plt.show()
    
    axisTableCombo = ""
    for id, (windA, windB, pearl) in enumerate(axisPair):
        pointsDist = pointToPointDist(windA, windB)
        distPearlToA = pointToPointDist(windA, pearl)
        distPearlToB = pointToPointDist(windB, pearl)
        pearlOffs = pointToLineDistance(pearl, windA, windB)
        colLbls = ["Property", "X", "Z"]
        rowNames = ["Point A", "Point B", "Pearl", "Length total", "Length to A", "Length to B", "Pearl offset from line"]
        cellVals = [windA, windB, pearl, (pointsDist, ""), (distPearlToA, ""), (distPearlToB, ""), (pearlOffs, "")]
        combinedRowData = [(descr, *vals) for descr, vals in zip(rowNames, cellVals)]
        axisTable = tabulate(combinedRowData, headers=colLbls, tablefmt="github", floatfmt=".15g")

        axisTableCombo += f"### **Axis {id+1}**" + "\r\n"
        axisTableCombo += axisTable + "\r\n"
    
    plotWidget = widgets.Output(layout={'margin': '20px', 'overflow': 'hidden'})
    with plotWidget:
        plt.show()
    tableWidget = widgets.Output()
    with tableWidget:
        display(Markdown(axisTableCombo))
    dashboard = widgets.HBox([plotWidget, tableWidget])
    display(dashboard)
    
        

selector = widgets.IntSlider(
    value=0,
    min=0,
    max=len(axisCandidates) - 1,
    step=1,
    description='View Index:',
    continuous_update=False
)

axisViewer = widgets.interact(plotResult, index=selector)

interactive(children=(IntSlider(value=0, continuous_update=False, description='View Index:', max=27), Output()…